<script async src="https://www.googletagmanager.com/gtag/js?id=UA-59152712-8"></script>
<script>
  window.dataLayer = window.dataLayer || [];
  function gtag(){dataLayer.push(arguments);}
  gtag('js', new Date());

  gtag('config', 'UA-59152712-8');
</script>

# [Brill-Lindquist](https://journals.aps.org/pr/abstract/10.1103/PhysRev.131.471) Initial data
## Author: Zach Etienne
###  Formatting improvements courtesy Brandon Clark

<font color='green'>**This module has been validated to exhibit convergence to zero of the Hamiltonian constraint violation at the expected order to the exact solution (see plot at bottom of [the exact initial data validation start-to-finish tutorial notebook](Tutorial-Start_to_Finish-BSSNCurvilinear-Setting_up_Exact_Initial_Data.ipynb); momentum constraint is zero), and all quantities have been validated against the [original SENR code](https://bitbucket.org/zach_etienne/nrpy).**</font>

### NRPy+ Source Code for this module: [BrillLindquist.py](../edit/BSSN/BrillLindquist.py)

## Introduction:
This module sets up initial data for a merging black hole system in Cartesian coordinates.

<a id='toc'></a>

# Table of Contents
$$\label{toc}$$ 

This notebook is organized as follows

1. [Step 1](#initializenrpy): Initialize core Python/NRPy+ modules
1. [Step 2](#initialdata): Setting up Brill-Lindquist initial data
1. [Step 3](#code_validation): Code Validation against BSSN/BrillLindquist NRPy+ module
1. [Step 4](#latex_pdf_output): Output this notebook to $\LaTeX$-formatted PDF file

<a id='initializenrpy'></a>

# Step 1: Initialize core Python/NRPy+ modules \[Back to [top](#toc)\]
$$\label{initializenrpy}$$

Let's start by importing all the needed modules from Python/NRPy+:

In [1]:
# Step 1: Initialize core Python/NRPy+ modules
import sympy as sp             # SymPy: The Python computer algebra package upon which NRPy+ depends
import NRPy_param_funcs as par # NRPy+: Parameter interface
import indexedexp as ixp       # NRPy+: Symbolic indexed expression (e.g., tensors, vectors, etc.) support

<a id='initialdata'></a>

# Step 2: Setting up Brill-Lindquist initial data \[Back to [top](#toc)\]
$$\label{initialdata}$$ 

Here we set up Brill-Lindquist initial data ([Brill & Lindquist, Phys. Rev. 131, 471, 1963](https://journals.aps.org/pr/abstract/10.1103/PhysRev.131.471); see also Eq. 1 of [Brandt & Brügmann, arXiv:gr-qc/9711015v1](https://arxiv.org/pdf/gr-qc/9711015v1.pdf)):

The conformal factor $\psi$ for Brill-Lindquist initial data is given by
$$\psi = e^{\phi} = 1 + \sum_{i=1}^N \frac{m_{(i)}}{2 \left|\vec{r}_{(i)} - \vec{r}\right|};\quad K_{ij}=0,$$

where $\psi$ is written in terms of the 3-metric $\gamma_{ij}$ as

$$
\gamma_{ij} = \psi^4 \delta_{ij}.
$$

The extrinsic curvature is zero:
$$
K_{ij} = 0
$$

These data consist of $N$ nonspinning black holes initially at rest. This module restricts to the case of two such black holes, positioned anywhere. Here, we implement $N=2$.

**Inputs for $\psi$**:
* The position and (bare) mass of black hole 1: $\left(x_{(1)},y_{(1)},z_{(1)}\right)$ and $m_{(1)}$, respectively
* The position and (bare) mass of black hole 2: $\left(x_{(2)},y_{(2)},z_{(2)}\right)$ and $m_{(2)}$, respectively

**Additional variables needed for spacetime evolution**:
* Desired coordinate system
* Desired initial lapse $\alpha$ and shift $\beta^i$. We will choose our gauge conditions as $\alpha=\psi^{-2}$ and $\beta^i=B^i=0$.

In [2]:
# Step 2: Setting up Brill-Lindquist initial data

thismodule = "Brill-Lindquist"
BH1_posn_x,BH1_posn_y,BH1_posn_z = par.Cparameters("REAL", thismodule,
                                                   ["BH1_posn_x","BH1_posn_y","BH1_posn_z"],
                                                   [         0.0,         0.0,        +0.5])
BH1_mass = par.Cparameters("REAL", thismodule, ["BH1_mass"],1.0)
BH2_posn_x,BH2_posn_y,BH2_posn_z = par.Cparameters("REAL", thismodule,
                                                   ["BH2_posn_x","BH2_posn_y","BH2_posn_z"],
                                                   [         0.0,         0.0,        -0.5])
BH2_mass = par.Cparameters("REAL", thismodule, ["BH2_mass"],1.0)

# Step 2.a: Set spatial dimension (must be 3 for BSSN)
DIM = 3
par.set_parval_from_str("grid::DIM",DIM)

Cartxyz = ixp.declarerank1("Cartxyz")

# Step 2.b: Set psi, the conformal factor:
psi = sp.sympify(1)
psi += BH1_mass / ( 2 * sp.sqrt((Cartxyz[0]-BH1_posn_x)**2 + (Cartxyz[1]-BH1_posn_y)**2 + (Cartxyz[2]-BH1_posn_z)**2) )
psi += BH2_mass / ( 2 * sp.sqrt((Cartxyz[0]-BH2_posn_x)**2 + (Cartxyz[1]-BH2_posn_y)**2 + (Cartxyz[2]-BH2_posn_z)**2) )

# Step 2.c: Set all needed ADM variables in Cartesian coordinates
gammaCartDD = ixp.zerorank2()
KCartDD     = ixp.zerorank2() # K_{ij} = 0 for these initial data
for i in range(DIM):
    gammaCartDD[i][i] = psi**4

alphaCart = 1/psi**2
betaCartU = ixp.zerorank1() # We generally choose \beta^i = 0 for these initial data
BCartU    = ixp.zerorank1() # We generally choose B^i = 0 for these initial data

<a id='code_validation'></a>

# Step 3: Code Validation against BSSN/BrillLindquist NRPy+ module  \[Back to [top](#toc)\]
$$\label{code_validation}$$

Here, as a code validation check, we verify agreement in the SymPy expressions for Brill-Lindquist initial data between

1. this tutorial and 
2. the NRPy+ [BSSN/BrillLindquist.py](../edit/BSSN/BrillLindquist.py) module.

In [3]:
# Step 3: Code Validation against BSSN.BrillLindquist NRPy+ module

# First we import reference_metric, which is
#   needed since BSSN.BrillLindquist calls
#   BSSN.ADM_Exact_Spherical_or_Cartesian_to_BSSNCurvilinear, which
#   depends on reference_metric:
import reference_metric as rfm   # NRPy+: Reference metric support
rfm.reference_metric()

import BSSN.BrillLindquist as bl
bl.BrillLindquist()

print("Consistency check between Brill-Lindquist tutorial and NRPy+ BSSN.BrillLindquist module. ALL SHOULD BE ZERO.")
print("alphaCart - bl.alphaCart = "+str(sp.simplify(alphaCart - bl.alphaCart)))
for i in range(DIM):
    print("betaCartU["+str(i)+"] - bl.betaCartU["+str(i)+"] = "+\
          str(sp.simplify(betaCartU[i] - bl.betaCartU[i])))
    print("BCartU["+str(i)+"] - bl.BaCartU["+str(i)+"] = "+str(sp.simplify(BCartU[i] - bl.BCartU[i])))
    for j in range(DIM):
        print("gammaCartDD["+str(i)+"]["+str(j)+"] - bl.gammaCartDD["+str(i)+"]["+str(j)+"] = "+\
              str(sp.simplify(gammaCartDD[i][j] - bl.gammaCartDD[i][j])))
        print("KCartDD["+str(i)+"]["+str(j)+"] - bl.KCartDD["+str(i)+"]["+str(j)+"] = "+\
              str(sp.simplify(KCartDD[i][j] - bl.KCartDD[i][j])))

Consistency check between Brill-Lindquist tutorial and NRPy+ BSSN.BrillLindquist module. ALL SHOULD BE ZERO.
alphaCart - bl.alphaCart = 0
betaCartU[0] - bl.betaCartU[0] = 0
BCartU[0] - bl.BaCartU[0] = 0
gammaCartDD[0][0] - bl.gammaCartDD[0][0] = 0
KCartDD[0][0] - bl.KCartDD[0][0] = 0
gammaCartDD[0][1] - bl.gammaCartDD[0][1] = 0
KCartDD[0][1] - bl.KCartDD[0][1] = 0
gammaCartDD[0][2] - bl.gammaCartDD[0][2] = 0
KCartDD[0][2] - bl.KCartDD[0][2] = 0
betaCartU[1] - bl.betaCartU[1] = 0
BCartU[1] - bl.BaCartU[1] = 0
gammaCartDD[1][0] - bl.gammaCartDD[1][0] = 0
KCartDD[1][0] - bl.KCartDD[1][0] = 0
gammaCartDD[1][1] - bl.gammaCartDD[1][1] = 0
KCartDD[1][1] - bl.KCartDD[1][1] = 0
gammaCartDD[1][2] - bl.gammaCartDD[1][2] = 0
KCartDD[1][2] - bl.KCartDD[1][2] = 0
betaCartU[2] - bl.betaCartU[2] = 0
BCartU[2] - bl.BaCartU[2] = 0
gammaCartDD[2][0] - bl.gammaCartDD[2][0] = 0
KCartDD[2][0] - bl.KCartDD[2][0] = 0
gammaCartDD[2][1] - bl.gammaCartDD[2][1] = 0
KCartDD[2][1] - bl.KCartDD[2][1] = 0
gammaCartDD

<a id='latex_pdf_output'></a>

# Step 4: Output this notebook to $\LaTeX$-formatted PDF file \[Back to [top](#toc)\]
$$\label{latex_pdf_output}$$

The following code cell converts this Jupyter notebook into a proper, clickable $\LaTeX$-formatted PDF file. After the cell is successfully run, the generated PDF may be found in the root NRPy+ tutorial directory, with filename
[Tutorial-ADM_Initial_Data-Brill-Lindquist](Tutorial-ADM_Initial_Data-Brill-Lindquist.pdf) (Note that clicking on this link may not work; you may need to open the PDF file through another means.)


In [4]:
import cmdline_helper as cmd    # NRPy+: Multi-platform Python command-line interface
cmd.output_Jupyter_notebook_to_LaTeXed_PDF("Tutorial-ADM_Initial_Data-Brill-Lindquist")

Created Tutorial-ADM_Initial_Data-Brill-Lindquist.tex, and compiled LaTeX
    file to PDF file Tutorial-ADM_Initial_Data-Brill-Lindquist.pdf
